In [2]:
def calculate_y31(point1, point2, x3_1):
    """
    根据给定的两个点和一个x值，计算对应的y3值并验证范围
    
    参数：
        point1: 第一个点的坐标，格式为 [x1, y1]
        point2: 第二个点的坐标，格式为 [x2, y2]
        x3_1: 需要求对应y3的x坐标
    
    返回：
        符合条件的y3值（float），若不符合条件则返回None
    """
    x1, y1 = point1
    x2, y2 = point2
    
    # 处理x1与x2重合的特殊情况（避免除零错误）
    if x1 == x2:
        # 垂直直线上只有x3等于x1时，y3取y1（但需额外判断是否在y1和y2之间）
        if x3_1 != x1:
            return None
        else:
            # 垂直直线上y3理论上可任意，
            return y2  # 垂直直线上取y2作为参考值

    
    # 计算直线斜率和y3值
    slope = (y2 - y1) / (x2 - x1)
    y3 = y1 + slope * (x3_1 - x1)
    
    # 判断y3是否在y1和y2之间（包含端点）
    if (y1 <= y3 <= y2) or (y2 <= y3 <= y1):
        return y3
    else:
        return None

In [3]:
def polygon_area(abs_coords):
    """
    计算多边形面积（鞋带公式）
    abs_coords: 顶点列表，格式为 [(x1,y1), (x2,y2), ..., (xn,yn)]，需闭合或自动补全
    """
    if not abs_coords:
        return (0, 0, 0, 0, 0)
    xs = [p[0] for p in abs_coords]
    ys = [p[1] for p in abs_coords]
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    n = len(abs_coords)
    area = 0.0
    for i in range(n):
        x_i, y_i = abs_coords[i]
        x_j, y_j = abs_coords[(i+1) % n]  # 最后一点连接第一点
        area += (x_i * y_j) - (x_j * y_i)
    return (min_x, min_y, max_x, max_y, abs(area) / 2.0)

In [4]:
def get_file_name(img_dir, txt_dir):
    # 获取img目录中所有文件的绝对路径
    img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
    
    # 获取txt目录中所有文件的绝对路径
    txt_files = [os.path.join(txt_dir, f) for f in os.listdir(txt_dir) if os.path.isfile(os.path.join(txt_dir, f))]
    
    return img_files, txt_files

In [5]:
def get_tiff_info(img):
    """获取TIFF的位深、压缩方式等关键信息"""
    # 检测位深（模式映射）
    mode_to_bits = {
        'L': 8,          # 8位灰度（无符号整数）
        'I;16': 16,      # 16位灰度（大端）
        'I;16B': 16,     # 16位灰度（小端，常见于Windows）
        'I;16S': 16      # 16位灰度（有符号整数，罕见但需兼容）
    }
    bit_depth = mode_to_bits.get(img.mode, None)
    if bit_depth is None:
        raise ValueError(f"不支持的TIFF模式: {img.mode}（仅支持8/16位灰度）")

    # 检测压缩方式（兼容Pillow参数）
    compression_code = img.info.get('compression', 1)  # 默认无压缩（代码1）
    compression_map = {
        1: None,         # 无压缩（TIFF Tag 259=1）
        5: 'tiff_lzw',   # LZW压缩（TIFF Tag 259=5）
        8: 'tiff_deflate' # Deflate压缩（TIFF Tag 259=8）
    }
    compression = compression_map.get(compression_code, None)

    return {
        "bit_depth": bit_depth,
        "compression": compression,
        "mode": img.mode  # 保留原始模式（如I;16B）
    }

In [30]:
def get_ww_wc(img_array):
    # 获取图像的窗宽窗位
    # 使用96%的像素范围（忽略最高2%和最低2%的异常值）
    lower_percentile = np.percentile(img_array, 5)
    upper_percentile = np.percentile(img_array, 95)
    # 计算窗宽窗位
    window_width = int(upper_percentile - lower_percentile)
    window_level = int((upper_percentile + lower_percentile) // 2)

    print("window_width:", window_width)

    return window_width, window_level

In [7]:
def get_abs_coords(txt_path, img_width, img_height):
    """
    解析标注文件并将相对坐标转换为绝对坐标
    
    参数:
    txt_path (str): 标注文件路径
    img_width (int): 图像宽度
    img_height (int): 图像高度
    
    返回:
    list: 成功时返回包含(classid, 绝对坐标列表)的列表
    bool: 失败时返回False
    """
    # print("正在解析标注文件...")
    annotations = []
    try:
        with open(txt_path, "r") as f:
            for line_idx, line in enumerate(f):
                line = line.strip()
                if not line:
                    continue
                
                parts = line.split()
                
                # 验证参数数量是否足够
                if len(parts) < 6:  # 至少需要classid + 3组坐标（x_rel,y_rel）
                    print(f"警告：第{line_idx+1}行参数不足（至少需要classid + 3组相对坐标），已跳过")
                    continue
                
                try:
                    classid = int(parts[0])
                    rel_coords = []
                    
                    # 解析相对坐标（每两个元素为一个(x_rel,y_rel)）
                    for i in range(1, len(parts), 2):
                        if i + 1 >= len(parts):
                            print(f"警告：第{line_idx+1}行坐标点不完整（奇数个参数），已跳过")
                            break
                        
                        x_rel = float(parts[i])
                        y_rel = float(parts[i+1])
                        
                        # 转换为绝对坐标
                        x_abs = round(x_rel * img_width, 6)
                        y_abs = round(y_rel * img_height, 6)
                        
                        # 验证坐标是否在图像范围内
                        if not (0 <= x_abs <= img_width and 0 <= y_abs <= img_height):
                            print(f"警告：第{line_idx+1}行坐标超出图像范围（{x_abs}, {y_abs}），已跳过")
                            break
                        
                        rel_coords.append((x_abs, y_abs))
                    
                    else:
                        # 所有坐标点解析成功
                        if len(rel_coords) >= 3:  # 至少需要3个点形成多边形
                            annotations.append((classid, rel_coords))
                        else:
                            print(f"警告：第{line_idx+1}行坐标点数不足（至少需要3个点），已跳过")
                
                except Exception as e:
                    print(f"警告：第{line_idx+1}行解析失败（{str(e)}），已跳过")
    
    except Exception as e:
        print(f"错误：标注文件读取失败 - {str(e)}")
        return False
    
    # 如果没有找到任何有效标注
    if not annotations:
        print("警告：未找到有效标注")
        return False
    
    # print(f"共解析到{len(annotations)}个有效目标标注")
    return annotations    

In [8]:
def find_matching_path(path_list, target_name):
    for path in path_list:
        if Path(path).stem == target_name:
            return path
    print(f"在txt目录下没有与 {target_name} 对应的txt文件")
    return None

In [9]:
def get_point(img_width, STEP):
    # 验证参数有效性
    if not isinstance(img_width, int) or img_width <= 0:
        print(f"错误：图像宽度必须为正整数（当前值：{img_width}）")
        return False
    
    if not isinstance(STEP, int) or STEP <= 0:
        print(f"错误：切割步长必须为正整数（当前值：{STEP}）")
        return False
    
    # print("正在计算切割块位置...")
    cut_positions = []
    current_start = 0
    
    try:
        while current_start < img_width:
            cut_positions.append(current_start)
            current_start += STEP
        
        # 验证结果有效性（至少生成一个切割块）
        if not cut_positions:
            print(f"错误：无法生成切割块（图像宽度={img_width}，步长={STEP}）")
            return False
        
        print(f"共生成{len(cut_positions)}个切割块，覆盖宽度范围：0~{img_width}px")
        return cut_positions
    
    except Exception as e:
        print(f"错误：计算切割位置时发生异常 - {str(e)}")
        return False

In [17]:
import cv2
import numpy as np

def enhance_image(src, sigma_of_gauss=25, fanwei=100):
    # 记录原始数据类型，用于最终输出转换
    original_dtype = src.dtype
    
    # 创建并初始化滤波模板:拉普拉斯核3x3
    kernel = np.zeros((3, 3), dtype=np.float32)
    kernel[1, 1] = 5.0
    kernel[0, 1] = -0.9
    kernel[1, 0] = -0.9
    kernel[1, 2] = -0.9
    kernel[2, 1] = -0.9
    
    # 应用锐化滤波，使用float32避免负值截断问题
    src_float = src.astype(np.float32)
    src_sharpened = cv2.filter2D(src_float, -1, kernel)
    
    # 确保所有值非负，防止log(0)
    src_non_neg = np.maximum(src_sharpened, 0)
    
    # 根据输入类型调整缩放比例
    if original_dtype == np.uint8:
        scale_factor = 255.0  # 8位图像最大值
    else:
        scale_factor = 65535.0  # 默认按16位处理
    
    # 转换为浮点型并计算对数，归一化到[1/scale_factor, 1]范围
    src_normalized = (src_non_neg + 1) / scale_factor  # +1避免log(0)
    src_log = np.log(src_normalized)
    
    # 计算入射光线强度及其对数
    L = cv2.GaussianBlur(src_normalized, (0, 0), sigma_of_gauss)
    L_log = np.log(L)
    
    # 计算反射分量
    R = cv2.subtract(src_log, L_log)
    
    # 处理可能的NaN/inf值
    R = np.nan_to_num(R, nan=0.0, posinf=1e6, neginf=-1e6)
    
    # 计算图像均值与标准差
    mean, std = cv2.meanStdDev(R)
    min_val = mean[0, 0] - fanwei * std[0, 0]
    max_val = mean[0, 0] + fanwei * std[0, 0]
    
    # 防止分母为0
    if max_val <= min_val:
        # 如果没有动态范围，返回全0或原始图像
        return np.zeros_like(src, dtype=original_dtype)
    
    # 将图像拉伸映射到[0, scale_factor]
    R_scaled = (R - min_val) / (max_val - min_val) * scale_factor
    R_clipped = np.clip(R_scaled, 0, scale_factor)
    
    # 转换回原始数据类型
    if original_dtype == np.uint8:
        dst = R_clipped.astype(np.uint8)
    else:
        dst = R_clipped.astype(np.uint16)
    
    return dst

In [26]:
import cv2
import numpy as np

def adaptive_enhance(src, sigma_of_gauss=25, fanwei=100):
    """
    自动适配8位/16位输入的图像增强函数（保持原始位深输出）
    
    参数:
        src (np.ndarray): 输入的8位或16位单通道灰度图像（CV_8U/CV_16U）
        sigma_of_gauss (int): 高斯模糊标准差，默认25
        fanwei (int): 归一化范围系数（均值±fanwei×标准差），默认100
    
    返回:
        np.ndarray: 增强后的图像（与输入同位深，CV_8U/CV_16U）
    """
    # 校验输入是否为单通道灰度图
    if src.ndim != 2:
        raise ValueError("输入图像必须为单通道灰度图")
    
    # 自动检测位深（8位或16位）
    if src.dtype == np.uint8:
        bit_depth = 8
        max_val = 255  # 原始数据最大值
        norm_denominator = 256.0  # 加1后归一化的分母（0-255 → 1-256 → 1/256~1）
    elif src.dtype == np.uint16:
        bit_depth = 16
        max_val = 65535  # 原始数据最大值
        norm_denominator = 65536.0  # 加1后归一化的分母（0-65535 → 1-65536 → 1/65536~1）
    else:
        raise TypeError("仅支持8位（CV_8U）或16位（CV_16U）单通道灰度图像")

    # 步骤1：拉普拉斯滤波（保持浮点运算）
    laplacian_kernel = np.array([
        [0,   -0.9,  0],
        [-0.9, 5.0, -0.9],
        [0,   -0.9,  0]
    ], dtype=np.float32)
    filtered_img = cv2.filter2D(src, cv2.CV_32F, laplacian_kernel)  # 输出为32位浮点

    # 步骤2：预处理（加1防止log(0)，并归一化到[0,1]）
    img_plus_1 = src.astype(np.float32) + 1  # 8位→1-256，16位→1-65536
    img_normalized = img_plus_1 / norm_denominator  # 归一化到[1/256, 1]（8位）或[1/65536, 1]（16位）

    # 步骤3：计算对数图像（log(归一化后的值)）
    img_log = cv2.log(img_normalized)  # 8位结果范围≈[-5.54, 0]，16位≈[-11.09, 0]

    # 步骤4：计算入射光强的高斯模糊和对数
    gaussian_blur = cv2.GaussianBlur(img_normalized, (0, 0), sigma_of_gauss)  # 高斯模糊（自动推导核尺寸）
    gaussian_log = cv2.log(gaussian_blur)  # 入射光强的对数

    # 步骤5：计算反射分量（R = log(原图) - log(高斯模糊图)）
    reflection = cv2.subtract(img_log, gaussian_log)  # 反射分量

    # 步骤6：计算反射分量的统计量（均值和标准差）
    mean_val, std_val = cv2.meanStdDev(reflection)
    mean_val = mean_val[0, 0]  # 单通道均值（float）
    std_val = std_val[0, 0]    # 单通道标准差（float）

    # 步骤7：确定归一化范围（防止除零错误）
    min_range = mean_val - fanwei * std_val
    max_range = mean_val + fanwei * std_val

    # 步骤8：归一化并拉伸到原始位深的动态范围
    if max_range <= min_range:
        # 极端情况（如全黑/全白），返回全0或全max_val
        enhanced_img = np.full_like(src, max_val if fanwei > 0 else 0, dtype=src.dtype)
    else:
        # 将反射分量从[min_range, max_range]映射到[0, max_val]
        normalized = (reflection - min_range) / (max_range - min_range)
        normalized = np.clip(normalized, 0, 1)  # 限制在有效范围内（防止浮点误差）
        enhanced_img = (normalized * max_val).astype(src.dtype)  # 转换为原始位深

    return enhanced_img

In [28]:
def get_enhance_img(cropped_img, is_bit):
    
    cropped_arrary = np.array(cropped_img)


    # dst = enhance_image(cropped_arrary)

    # 验证图像实际位数与传入参数是否一致
    expected_dtype = np.uint16 if is_bit == 16 else np.uint8
    if cropped_arrary.dtype != expected_dtype:
        print(f"警告：图像实际位数（{cropped_arrary.dtype}）与传入的is_bit参数（{is_bit}）不匹配")

    is_16bit = (is_bit == 16)
    input_max = 65535 if is_16bit else 255  # 输入图像的最大像素值
    output_max = 65535 if is_16bit else 255  # 输出图像的最大像素值
    bit_depth = f"{is_bit}位"

    # 根据图像特性自动设置窗宽窗位
    window_width, window_level = get_ww_wc(cropped_arrary)
    
    # 应用窗宽窗位进行对比度拉伸
    window_min = max(0, window_level - window_width // 2)
    window_max = min(input_max, window_level + window_width // 2)

    # 像素值映射
    enhanced_array = np.clip(cropped_arrary, window_min, window_max)
    enhanced_array = (enhanced_array - window_min) / (window_max - window_min) * output_max
    enhanced_array = enhanced_array.astype(np.uint16 if is_16bit else np.uint8)

    dst = enhance_image(enhanced_array)

    enhanced_img = Image.fromarray(dst)

    return enhanced_img


In [12]:
img_dir = r"D:\object_file\ultralytics-seg\input\img"
txt_dir = r"D:\object_file\ultralytics-seg\input\txt"
output_img_dir = r"D:\object_file\ultralytics-seg\input\img-img"
output_txt_dir = r"D:\object_file\ultralytics-seg\input\txt-txt"

In [13]:
CROP_LENGTH = 640       # 横向切割长度（像素）
OVERLAP = 200           # 相邻切割块重叠像素数
STEP = CROP_LENGTH - OVERLAP  # 切割步长（像素）

In [14]:
import os
img_files, txt_files = get_file_name(img_dir, txt_dir)
print(img_files)

['D:\\object_file\\ultralytics-seg\\input\\img\\A_DJ-RT-20220622-42.tif', 'D:\\object_file\\ultralytics-seg\\input\\img\\A_DJ-RT-20220623-11.tif', 'D:\\object_file\\ultralytics-seg\\input\\img\\A_DJ-RT-20220708-35.tif', 'D:\\object_file\\ultralytics-seg\\input\\img\\A_DJ-RT-20220711-25.tif', 'D:\\object_file\\ultralytics-seg\\input\\img\\A_DJ-RT-20220725-1.tif']


In [15]:
from pathlib import Path
from tqdm import tqdm 
from PIL import Image 
import numpy as np

In [31]:
for img_path in img_files:
    tif_name = Path(img_path).stem
    # print(f"正在读取原始 {tif_name} 图像...")
    try:
        img = Image.open(img_path)
        # if img.mode != "L":  # 确保为单通道灰度模式
        #     img = img.convert("L")
        img_width, img_height = img.size
        print(f"原始图像尺寸：宽度={img_width}px，高度={img_height}px")
    except Exception as e:
        raise RuntimeError(f"图像读取失败：{str(e)}")
    
    # 获取原始TIFF的关键信息
    tiff_info = get_tiff_info(img)

    is_bit = tiff_info["bit_depth"]
    # print("is_bit:", is_bit)

    # 保存时匹配原始参数（位深+压缩方式）
    saved_params = {
        "format": "TIFF",
        "bits": tiff_info["bit_depth"],          # 动态设置位深（8或16）
        "compression": tiff_info["compression"]  # 动态设置压缩方式
    }

    txt_name = find_matching_path(txt_files, tif_name)
    if txt_name is None:
        continue

    annotations = get_abs_coords(txt_name, img_width, img_height)
    if not annotations:
        continue

    cut_positions = get_point(img_width, STEP)
    if not cut_positions:
        continue

    for block_idx, start_x in enumerate(tqdm(cut_positions)):

        # 当前切割块的绝对坐标范围（x: start_x → end_x，y: 0 → img_height）
        end_x = start_x + CROP_LENGTH
        end_x = min(end_x, img_width)  # 限制右边界
        block_width = end_x - start_x

        # 提取子图像（绝对位置裁剪）
        sub_img = img.crop((start_x, 0, end_x, img_height))
        cropped_img = sub_img.convert(tiff_info["mode"])  # 关键：保留原始模式

        enhanced_img = get_enhance_img(cropped_img, is_bit)
        save_img_path = os.path.join(output_img_dir, f"{tif_name}_block_{block_idx}.tif")
        enhanced_img.save(save_img_path, **saved_params)

        # 处理当前块的标注（绝对坐标判断）
        block_anns = []

        # 按行进行遍历，格式为 (classid, [(x1, y1), (x2, y2), (x3, y3), (x4, y4), (),,,])
        for ann in annotations:
            classid, abs_coords = ann
            if not abs_coords:  # 跳过无坐标点的无效目标
                continue
            
            # 定位目标的最小外接矩形，x_min y_min x_max y_max  
            orig_min_x, orig_min_y, orig_max_x, orig_max_y, orig_area = polygon_area(abs_coords)
            if orig_area == 0:
                continue  # 面积为0的目标无意义
            
            # 当前切割区域的最大内接矩形， x_min y_min x_max y_max  
            block_min_x, block_min_y = start_x, 0
            block_max_x, block_max_y = end_x, img_height

            # 元组转换为列表（list），进行遍历
            abs_coords_copy = [list(point) for point in abs_coords]
            abs_coords_copy.append(abs_coords_copy[0])

            # 复制一个列表记录改变的数据
            abs_coords_copy2 = [point.copy() for point in abs_coords_copy]
            
            # 没有交集
            if orig_max_x <= block_min_x or orig_min_x >= block_max_x:
                continue

            # 有交集 
            for i in range(len(abs_coords_copy)):
                current_point = abs_coords_copy[i]

                if i < len(abs_coords_copy) - 1:
                    next_point = abs_coords_copy[i+1]

                    if block_min_x <= current_point[0] <= block_max_x and block_min_x <= next_point[0] <= block_max_x:
                        continue
                    if current_point[0] >= block_max_x and block_min_x <= next_point[0] <= block_max_x:
                        y3 = calculate_y31(current_point, next_point, block_max_x)
                        abs_coords_copy2[i][0] = block_max_x
                        abs_coords_copy2[i][1] = y3
                    if next_point[0] >= block_max_x and block_min_x <= current_point[0] <= block_max_x:
                        y3 = calculate_y31(current_point, next_point, block_max_x)
                        abs_coords_copy2[i+1][0] = block_max_x
                        abs_coords_copy2[i+1][1] = y3
                    if block_min_x <= current_point[0] <= block_max_x and next_point[0] <= block_min_x:
                        y3 = calculate_y31(current_point, next_point, block_min_x)
                        abs_coords_copy2[i+1][0] = block_min_x
                        abs_coords_copy2[i+1][1] = y3
                    if block_min_x <= next_point[0] <= block_max_x and current_point[0] <= block_min_x:
                        y3 = calculate_y31(current_point, next_point, block_min_x)
                        abs_coords_copy2[i][0] = block_min_x
                        abs_coords_copy2[i][1] = y3 
                    if current_point[0] >= block_max_x and next_point[0] <= block_min_x:
                        y31 = calculate_y31(current_point, next_point, block_min_x)
                        y32 = calculate_y31(current_point, next_point, block_max_x)
                        abs_coords_copy2[i+1][0] = block_min_x
                        abs_coords_copy2[i+1][1] = y31
                        abs_coords_copy2[i][0] = block_max_x
                        abs_coords_copy2[i][1] = y32
                    if  next_point[0] >= block_max_x and current_point[0] <= block_min_x:
                        y31 = calculate_y31(current_point, next_point, block_min_x)
                        y32 = calculate_y31(current_point, next_point, block_max_x)
                        abs_coords_copy2[i+1][0] = block_max_x
                        abs_coords_copy2[i+1][1] = y32
                        abs_coords_copy2[i][0] = block_min_x
                        abs_coords_copy2[i][1] = y31


            # 根据x坐标筛选当前切割区域内的点
            abs_coords_copy3 = [point for point in abs_coords_copy2 if block_min_x <= point[0] <= block_max_x]

            if len(abs_coords_copy3) == 0:
                continue
            else:
                # 将坐标转换为相对坐标
                for j in abs_coords_copy3:
                    if j[0] == block_min_x:
                        j[0] = 0
                    else:
                        j[0] = j[0] - block_min_x
                    j[0] = j[0] / (block_max_x - block_min_x)
                    j[1] = j[1] / (block_max_y - block_min_y)
                    
                txt_data = [classid] + [item for sublist in abs_coords_copy3 for item in sublist]

                block_anns.append(txt_data)

        # 保存当前块的标注文件（相对坐标格式：classid x1_rel y1_rel x2_rel y2_rel ...）
        if block_anns:
            txt_path = os.path.join(output_txt_dir, f"{tif_name}_block_{block_idx}.txt")
            with open(txt_path, "w") as f:
                for ann in block_anns:
                    data_str = ' '.join(map(str, ann)) + '\n'
                    f.write(data_str)
        else:
            print(f"提示：块{block_idx}无有效目标（保留区域>10%），未生成标注文件")


原始图像尺寸：宽度=7833px，高度=725px
共生成18个切割块，覆盖宽度范围：0~7833px


  0%|          | 0/18 [00:00<?, ?it/s]

 11%|█         | 2/18 [00:00<00:01, 13.64it/s]

window_width: 12593
window_width: 12336
window_width: 14392


 33%|███▎      | 6/18 [00:00<00:00, 14.73it/s]

...

window_width: 9252
window_width: 8995
window_width: 11051


 78%|███████▊  | 14/18 [00:00<00:00, 18.86it/s]

window_width: 10537


100%|██████████| 18/18 [00:00<00:00, 19.42it/s]

window_width: 7967
window_width: 8481
window_width: 9509
